***********************************************************************************************************************************
    Language Translator Model: English to Hindi.
***********************************************************************************************************************************

In [1]:
#Installing all the required packages.
!pip install huggingface_hub
!pip install transformers
!pip install datasets 
!pip install --upgrade --force-reinstall pyarrow
!pip install transformers sentencepiece

  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/f3/94/4e2a579bbac1adb19e63b054b300f6f7fa04f32f212ce86c18727bdda698/pyarrow-16.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for numpy>=1.16.6 from https://files.pythonhosted.org/packages/1a/2e/151484f49fd03944c4a3ad9c418ed193cfd02724e138ac8a9505d056c582/numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (114 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 6.3 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp311-cp311-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.0.0
    Uninstalling pyarrow-16.0.0:
      Successfully uninstalled pyarrow-16.0.0
ERROR: pip's d

In [2]:
#importing the required packages.

import datasets         
import torch           
import transformers     

In [3]:
from datasets import load_dataset

#Loading the dataset from Hugging Face.
dataset = load_dataset('Ketan3101/English-Hindi-Translation')

#Accessing the 'train' split.
train_data = dataset['train']

#Initializing the variables to collect translations.
english_translation = None
hindi_translation = None

#Iterating through the dataset and extracting the translations.
for idx, example in enumerate(train_data):
    if example == 'english':
        #Skipping the 'english' marker and continue to the next item.
        continue
    elif example == 'hindi':
        #If encountering 'hindi' marker, printing the collected translations.
        if english_translation and hindi_translation:
            print(f"English: {english_translation}")
            print(f"Hindi: {hindi_translation}")
            print("-" * 20)
            english_translation = None
            hindi_translation = None
    else:
        #Collectting the English and Hindi translations.
        if english_translation is None:
            english_translation = example
        elif hindi_translation is None:
            hindi_translation = example

#Printing the last collected translations (if any).
if english_translation and hindi_translation:
    print(f"English: {english_translation}")
    print(f"Hindi: {hindi_translation}")
    print("-" * 20)


Found cached dataset parquet (/Users/db/.cache/huggingface/datasets/Ketan3101___parquet/Ketan3101--English-Hindi-Translation-d3447c8bcbbfe1e0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

English: {'english': 'Police have released a sketch of one of the militants believed to be in the city.', 'hindi': 'पुलिस ने आतंकवादियों में से एक का रेखाचित्र जारी किया है जिसके बारे में माना जा रहा है कि वह शहर में है।'}
Hindi: {'english': 'Police indicated that he had taken his own life.', 'hindi': 'पुलिस ने संकेत दिया कि उसने अपनी जान ले ली है।'}
--------------------


In [12]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

#Loading the dataset from Hugging Face.
dataset = load_dataset('Ketan3101/English-Hindi-Translation')

#Accessing the train dataset.
train_dataset = dataset['train']

#Converting to pandas DataFrame.
train_df = train_dataset.to_pandas()

#Splitting the DataFrame into training 80% and remaining data 20%.
train_data, remaining_data = train_test_split(train_df, test_size=0.2, random_state=42)

#Splitting the remaining data into validation 50% and test 50%.
val_data, test_data = train_test_split(remaining_data, test_size=0.5, random_state=42)

#Printing the sizes of each split.
print(f"Training examples: {len(train_data)}")
print(f"Validation examples: {len(val_data)}")
print(f"Test examples: {len(test_data)}")


Found cached dataset parquet (/Users/db/.cache/huggingface/datasets/Ketan3101___parquet/Ketan3101--English-Hindi-Translation-d3447c8bcbbfe1e0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Training examples: 279014
Validation examples: 34877
Test examples: 34877


In [52]:
from transformers import MarianTokenizer, MarianMTModel

def translate_english_to_hindi(input_text):
    #Model name for English to Hindi translation.
    model_name = "Helsinki-NLP/opus-mt-en-hi"

    #Loading tokenizer and model.
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    #Tokenizing the input text.
    inputs = tokenizer(input_text, return_tensors="pt")

    #Generating the translation with maximum new tokens limit.
    translated_ids = model.generate(**inputs, max_length=128)

    #Decoding the translated sequence into text.
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)

    return translated_text

#Taking input from the user as English Text.
english_input = input("Enter English text: ")

#Calling the translation function with the user's input.
hindi_output = translate_english_to_hindi(english_input)

#Delivering the Hindi translation.
print("Hindi translation:", hindi_output)


Enter English text:  I love my country


Hindi translation: मैं अपने देश से प्यार


In [48]:
import torch
from torch.nn import CrossEntropyLoss

#Defining a function to calculate the Perplexity of the model.
def calculate_perplexity(model, test_data):
    total_loss = 0.0
    total_tokens = 0

    for example in test_data:
        english_ids = torch.tensor(example['English']).unsqueeze(0).to(model.device)
        hindi_ids = torch.tensor(example['Hindi']).unsqueeze(0).to(model.device)

        with torch.no_grad():
            logits = model(input_ids=english_ids, labels=hindi_ids)[1]
            loss_fn = CrossEntropyLoss(reduction='sum')
            batch_loss = loss_fn(logits.view(-1, logits.size(-1)), hindi_ids.view(-1))

        total_loss += batch_loss.item()
        total_tokens += hindi_ids.size(1)

    avg_loss = total_loss / len(test_data)
    perplexity = torch.exp(torch.tensor(avg_loss)) 

    return perplexity.item()

#Sample data collection from the test split.
test_data = [
    {'English': [1, 123, 456, 789], 'Hindi': [1, 321, 654, 987]},  
    {'English': [1, 234, 567, 890], 'Hindi': [1, 432, 765, 98]}
]

model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#Calculating perplexity on the test set
test_perplexity = calculate_perplexity(model, test_data)
print(f"Perplexity on Test Set Translations: {test_perplexity}")


Perplexity on Test Set Translations: 2.3498596787621384e+24
